# This notebook is to assess document sign and other conventions

In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [2]:
import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

plt.rcParams['text.usetex'] = False # Plotting the global model is much much faster with tex fonts turned off

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)

# Let's plot the index for each segment to make choosing segments easier

In [72]:
lon_min=235
lon_max=255
lat_min=30
lat_max=50

plt.figure(figsize=(12, 12))
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], str(i), color="r", clip_on=True, fontsize=6, horizontalalignment="center", verticalalignment="center")
# plt.xlim([lon_min, lon_max])
# plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("segment indices")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Let's pick a segment to visualize

In [84]:
SEGMENT_IDX = 613
segment.dip.values[SEGMENT_IDX] = 45
segment.locking_depth.values[SEGMENT_IDX] = 50

In [85]:
# strike-slip
celeri.plot_segment_displacements(segment, station, command, segment_idx=SEGMENT_IDX, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
# dip-slip
celeri.plot_segment_displacements(segment, station, command, segment_idx=SEGMENT_IDX, strike_slip=0, dip_slip=1, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
# tensile-slip
celeri.plot_segment_displacements(segment, station, command, segment_idx=SEGMENT_IDX, strike_slip=0, dip_slip=0, tensile_slip=1, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Summary of Okada slip rate conventions:
- ## Locking depth is positive down  
| type | sign | interpretation |
| - | - | - |
| strike-slip | positive | left-lateral |
| strike-slip | negative | right-lateral |
| dip-slip | positive | convergence |
| dip-slip | negative | extension |
| tensile-slip | positive | extension |
| tensile-slip | negative | convergence |
- ## Note: The difference in sign for convergence and extension for dip-slip and tensile-slip is not idea but it seems consistent with Okada?  Should I change this so that it is more intiutive (e.g., positive numbers are always convergence) or leave as is for consistency with Okada?


# Now let's take a look at differential block velocities

In [73]:
SEGMENT_IDX = 65
segment.dip.values[SEGMENT_IDX] = 45
segment.locking_depth.values[SEGMENT_IDX] = 50


assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly
# operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)

# Build all linear kinematic operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)

assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [74]:
# Rotations only - with JDF a priori and no global float
data_vector = np.zeros(2 * assembly.data.n_stations
                     + 3 * assembly.data.n_block_constraints)
data_vector[0:2 * assembly.data.n_stations] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)

# Add block motion costraints to data vector
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = assembly.data.block_constraints
data_vector[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints] = 0

operator = np.zeros((2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, 3 * len(block)))
operator[0:2 * assembly.data.n_stations, :] = np.delete(operators.block_rotation, np.arange(2, operators.block_rotation.shape[0], 3), axis=0) # Delete up velocity partials
operator[2 * assembly.data.n_stations:2 * assembly.data.n_stations + 3 * assembly.data.n_block_constraints, :]  = operators.block_motion_constraints

estimation = addict.Dict()
state_vector = np.linalg.inv(operator.T @ operator) @ operator.T @ data_vector 
estimation.predictions = operator @ state_vector
vel = estimation.predictions[0:2 * assembly.data.n_stations]
estimation.east_vel = vel[0::2]
estimation.north_vel = vel[1::2]
east_vel_rotation_only = vel[0::2]
north_vel_rotation_only = vel[1::2]

# Plot observed and estimated velocities
lon_min=235
lon_max=255
lat_min=30
lat_max=50
quiver_scale=1e2
plt.figure()
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
# plt.quiver(station.lon, station.lat, station.east_vel, station.north_vel, scale=quiver_scale, scale_units="inches", color="g")
plt.quiver(station.lon, station.lat, estimation.east_vel, estimation.north_vel, scale=quiver_scale, scale_units="inches", color="r")
# plt.xlim([lon_min, lon_max])
# plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("observed and estimated velocities - block rotations only")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Convert estimated block motions to fault slip rates

In [83]:
estimated_slip_rates = operators.slip_rate_segment_block @ state_vector
estimated_strike_slip_rates = estimated_slip_rates[0::3]
estimated_dip_slip_rates = estimated_slip_rates[1::3]
estimated_tensile_slip_rates = estimated_slip_rates[2::3]

plt.figure(figsize=(12, 12))
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{estimated_strike_slip_rates[i]:.1f}", color="r", clip_on=True, fontsize=6, horizontalalignment="center", verticalalignment="center")
plt.xlim([lon_min, lon_max])
plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("strike-slip rates")
plt.show()


plt.figure(figsize=(12, 12))
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{estimated_dip_slip_rates[i]:.1f}", color="r", clip_on=True, fontsize=6, horizontalalignment="center", verticalalignment="center")
# plt.xlim([lon_min, lon_max])
# plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("dip-slip rates")
plt.show()


plt.figure(figsize=(12, 12))
for i in range(len(segment)):
    plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-k", linewidth=0.5)
    plt.text(segment.mid_lon_plate_carree[i], segment.mid_lat_plate_carree[i], f"{estimated_tensile_slip_rates[i]:.1f}", color="r", clip_on=True, fontsize=6, horizontalalignment="center", verticalalignment="center")
# plt.xlim([lon_min, lon_max])
# plt.ylim([lat_min, lat_max])
plt.gca().set_aspect("equal", adjustable="box")
plt.title("tensile-slip rates")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Summary of differential block motion/slip rate conventions:
| type | sign | interpretation |
| - | - | - |
| strike-slip | positive | left-lateral |
| strike-slip | negative | right-lateral |
| dip-slip | positive | convergence |
| dip-slip | negative | extension |
| tensile-slip | positive | extension |
| tensile-slip | negative | convergence |

- ## Note: The difference in sign for convergence and extension for dip-slip and tensile-slip is not ideal but it seems consistent with Okada.  Should I change this so that it is more intiutive (e.g., positive numbers are always convergence) or leave as is for consistency with Okada?  To make the tensile slip rate consistent with Okada there is a scale_factor = -1 in celeri.get_fault_slip_rate_partials.